<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_V20_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 0

Run date time (IST): 2024-01-26 11:15:01


In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspectsScrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','EQUITAS', 'SCHAEFFLER ', 'ADANITRANS']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
#stock_ids

In [4]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()

In [5]:
import yfinance as yfin
def get_stock_v20(stock_symbol):
    print(stock_symbol)
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Candle'] = 'R'
    stock_df.loc[stock_df['Close'] > stock_df['Open'], 'Candle'] = 'G'
    stock_df['Prev_Candle'] = stock_df['Candle'].shift(1)
    stock_df['Next_Candle'] = stock_df['Candle'].shift(-1)
    stock_df['Signal'] = 'NA'
    stock_df.loc[(stock_df['Candle'] != stock_df['Prev_Candle']) & (stock_df['Candle'] == 'G'), 'Signal'] = 'Start'
    stock_df.loc[(stock_df['Candle'] != stock_df['Next_Candle']) & (stock_df['Next_Candle'] == 'R') & (stock_df['Signal'] == 'NA'), 'Signal'] = 'End'
    stock_df.loc[(stock_df['Candle'] != stock_df['Next_Candle']) & (stock_df['Next_Candle'] == 'R') & (stock_df['Signal'] == 'Start'), 'Signal'] = 'Start/End'
    stock_df = stock_df[stock_df['Signal'] != 'NA']
    stock_df['Prev_Low'] = stock_df['Low'].shift(1)
    stock_df = stock_df[stock_df['Signal'] != 'Start']
    stock_df.loc[(stock_df['Signal'] == 'End'), 'Low'] = stock_df['Prev_Low']
    stock_df['Up%'] = round((stock_df['High'] - stock_df['Low'])*100/stock_df['Low'],2)
    stock_df = stock_df[stock_df['Up%'] > 20]
    stock_df['Current'] = yfin.Ticker(stock_symbol).history(period='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]['Close'].values[0]
    stock_df = stock_df[stock_df['Current'] < stock_df['Low']]
    stock_df['Stock'] = stock_symbol
    return stock_df

def get_v20():
    df_v20 = pd.DataFrame()
    for stock_id in stock_ids:
        #print(stock_id)
        df_tmp = get_stock_v20(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        df_v20 = df_v20.append(df_tmp)

    return df_v20.sort_index()

In [6]:
cols = ['Stock', 'Up%', '200_SMA', 'Close' ,'Criteria']
df_v20_prospects = get_v20()[cols]

3MINDIA.NS
5PAISA.NS
AARTIIND.NS
AAVAS.NS
ABBOTINDIA.NS
ABSLAMC.NS
ACC.NS
ADANIENT.NS
ADANIGREEN.NS
ADANIPORTS.NS
AKZOINDIA.NS
ALKYLAMINE.NS
AMBUJACEM.NS
ANGELONE.NS
APEX.NS
APOLLOHOSP.NS
ARE&M.NS
ASIANPAINT.NS
ASIANTILES.NS
ASTRAZEN.NS
AUBANK.NS
AUROPHARMA.NS
AVANTIFEED.NS
AXISBANK.NS
BAJAJ-AUTO.NS
BAJAJFINSV.NS
BAJAJHIND.NS
BAJAJHLDNG.NS
BAJFINANCE.NS
BANDHANBNK.NS
BANKBARODA.NS
BANKINDIA.NS
BATAINDIA.NS
BAYERCROP.NS
BERGEPAINT.NS
BHARTIARTL.NS
BIOCON.NS
BIRLAMONEY.NS
BLUESTARCO.NS
BOSCHLTD.NS
BPCL.NS
BRITANNIA.NS
CAMPUS.NS
CAMS.NS
CANBK.NS
CAPLIPOINT.NS
CASTROLIND.NS
CDSL.NS
CERA.NS
CHOLAFIN.NS
CIPLA.NS
CLEAN.NS
COALINDIA.NS
COLPAL.NS
CROMPTON.NS
CUB.NS
DABUR.NS
DEEPAKNTR.NS
DEN.NS
DIVISLAB.NS
DIXON.NS
DLF.NS
DMART.NS
DREAMFOLKS.NS
DRREDDY.NS
EICHERMOT.NS
EMAMILTD.NS
EQUITASBNK.NS
ERIS.NS
FEDERALBNK.NS
FINCABLES.NS
FINEORG.NS
GAIL.NS
GICRE.NS
GILLETTE.NS
GLAND.NS
GLAXO.NS
GODREJCP.NS
GODREJIND.NS
GRASIM.NS
GREENPANEL.NS
GULFOILLUB.NS
HAPPSTMNDS.NS
HATHWAY.NS
HAVELLS.NS
HCLTECH.NS
HD

In [7]:
df_v20_prospects_200SMA = df_v20_prospects[(df_v20_prospects['Close'] <= df_v20_prospects['200_SMA'])]

In [8]:
df_v20_prospects_V40N = df_v20_prospects[(df_v20_prospects['Criteria'] == 'V40N') | (df_v20_prospects['Criteria'] == 'V40')]

In [9]:
pd.concat([df_v20_prospects_V40N, df_v20_prospects_200SMA])

,Stock,Up%,200_SMA,Close,Criteria
Date,,,,,
2021-02-02 00:00:00+05:30,INDIGOPNTS.NS,28.45,3104.0,3104.490479,V40N
2021-03-02 00:00:00+05:30,SYMPHONY.NS,40.43,1035.0,1413.568726,V40N
2021-04-06 00:00:00+05:30,LALPATHLAB.NS,24.44,2403.0,3017.052979,V40N
2021-05-28 00:00:00+05:30,LUXIND.NS,61.36,1854.0,3233.427734,V40N
2021-10-27 00:00:00+05:30,TTKPRESTIG.NS,23.52,809.0,1097.593018,V40N
2021-11-18 00:00:00+05:30,SFL.NS,20.50,1122.0,1593.900024,V40N
2022-01-12 00:00:00+05:30,SFL.NS,21.16,1237.0,1882.724976,V40N
2022-01-27 00:00:00+05:30,LUXIND.NS,25.10,3472.0,2787.197266,V40N
2022-02-03 00:00:00+05:30,TEAMLEASE.NS,20.78,4015.0,4627.649902,V40N


In [10]:
df_v20_prospects['Criteria'].value_counts()

VR      25
V40N    11
AR       9
VPS      2
V40      1
Name: Criteria, dtype: int64